# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI

from langchain.prompts.prompt import PromptTemplate

llm = OpenAI(temperature=0.7)

get_name = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["gender"], template="Give me a totally random full name for a {gender}:"),
    n=2,
)
# create_draft_answer_chain.run("woman")

get_initials = LLMChain(
    llm=llm,
    prompt=PromptTemplate(input_variables=["name"], template="The initials of the name '{name}' are"),
    n=2,
)

overall_chain = SimpleSequentialChain(chains=[get_name, get_initials], verbose=True)
overall_chain.run("woman")



> Entering new SimpleSequentialChain chain...
selected_inputs
{'gender': 'woman'}
list_of_selected_inputs
[['woman']]
list_of_branches:
[{'gender': 'woman'}]
responses
['\n\nMarilyn Lorraine Richardson', '\n\nKathryn Matilda Montgomery']
['\n\nMarilyn Lorraine Richardson', '\n\nKathryn Matilda Montgomery']
selected_inputs
{'name': ['\n\nMarilyn Lorraine Richardson', '\n\nKathryn Matilda Montgomery']}
list_of_selected_inputs
[['\n\nMarilyn Lorraine Richardson', '\n\nKathryn Matilda Montgomery']]
list_of_branches:
[{'name': '\n\nMarilyn Lorraine Richardson'}, {'name': '\n\nKathryn Matilda Montgomery'}]
responses
[' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.']
[' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.']

> Finished SimpleSequentialChain chain.


[' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.', ' MLR and KMM.']